In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
import imutils
import sys

In [30]:
def autocenter(bbox, pad={'r':100, 'l':50, 't':50, 'b':100}, ratio=1):
    x,y,w,h=bbox
    ratio_real = h/w
    if w>h:
        hn = int(w * ratio)
        wn = w
    else:
        hn = h
        wn = int(h * ratio)
    
    yn=(y + (h//2)) - (hn//2)
    xn=(x + (w//2)) - (wn//2)
    xn = xn - pad['l']
    yn = yn - pad['t']
    hn = hn + pad['b']
    wn = wn + pad['r']
    
    return (xn, yn, wn, hn)

# def autocenter(bbox, new_size=(500,500)):
#     x,y,h,w = bbox
#     hn = new_size[0]
#     wn = new_size[1]

#     xc = x + 1/2*w
#     yc = y + 1/2*h

#     hc = 1/2hn - 1/2h
#     wc = 1/2wn - 1/2w

#     xn = wc - xc
#     yn = hc - yc

#     return xn,yn,hn,wn

In [31]:
cap = cv2.VideoCapture(0)

while True: 
    _, frame = cap.read()
    
    blurred_frame = cv2.GaussianBlur(frame, (15, 15), 10)
    hsv = cv2.cvtColor(blurred_frame, cv2.COLOR_BGR2HSV)

    lower = np.array([50, 100, 100])
    upper = np.array([255, 255, 255])

    mask = cv2.inRange(hsv, lower, upper)

    mask_inv = cv2.bitwise_not(mask)
    im, cntr, hierarcy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# find contours of large enough area
    min_coin_area = 600
    large_contours = [cnt for cnt in cntr if cv2.contourArea(cnt) > min_coin_area]
# large_contours    

# draw contours
#     cv2.drawContours(frame, large_contours, -1, (0, 255, 0), 2)
    sys.stdout.write('\r count: '+str(len(large_contours)))
    
    for idx in range(len(large_contours)):
        bbox = cv2.boundingRect(large_contours[idx])
        x,y,w,h=bbox
        x1,y1,w1,h1=autocenter(bbox)
#         cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255),2)
        cv2.rectangle(frame, (x1,y1), (x1+w1, y1+h1), (0,255,0),2)
#         if y1+h1>y1 and x1+w1>x1:
#             plt.imshow(frame[y1:y1+h1, x1:x1+w1])
#             cv2.imshow('frame1', frame[y1:y1+h1,x1:x1+w1])
        
    cv2.imshow('frame', frame)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

 count: 0